In [ ]:
from coiled import Cluster
from dask.distributed import Client

In [ ]:
cluster = Cluster(n_workers=50, package_sync=True)

In [ ]:
csv_dir = "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2/"
output_dir = "s3://coiled-datasets/h2o-benchmark/pyarrow_strings/N_1e9_K_1e2/"

In [ ]:
client = Client(cluster)
client

In [ ]:
import dask.dataframe as dd
import os

In [ ]:
ddf = dd.read_csv(
        os.path.join(csv_dir, "*.csv"),
        dtype={
            "id1": "string[pyarrow]", 
            "id2": "string[pyarrow]",
            "id3": "string[pyarrow]",
            "id4": "Int32",
            "id5": "Int32",
            "id6": "Int32",
            "v1": "Int32",
            "v2": "Int32",
            "v3": "float64",
        },
    )

In [ ]:
%%time
ddf.repartition(partition_size="100MB").to_parquet(
        output_dir,
        engine="pyarrow",
        compression="snappy",
        write_metadata_file=False,
    )

### NOTE
When using this dataset, rememeber that you need to read the columns casting `pyarrow[strings]` and the size of partitions is checked doing.

```python
ddf = dd.read_parquet("uri_of_pq_dir")
mem = ddf.astype({"id1": "string[pyarrow]",
                  "id2": "string[pyarrow]", 
                  "id3": "string[pyarrow]"}).memory_usage_per_partition(deep=True).compute()

from dask.utils import format_bytes
#check one partition
format_bytes(mem[0]) #this should be approximately 100MB
```

In [ ]:
cluster.close()

In [ ]:
client.shutdown()

In [ ]:
from dask.distributed import Client

client = Client()
client

In [2]:
import dask.dataframe as dd
from dask.utils import format_bytes

ddf = dd.read_parquet("s3://coiled-datasets/h2o-benchmark/pyarrow_strings/N_1e9_K_1e2/part.0.parquet")

mem = ddf.astype({"id1": "string[pyarrow]",
                  "id2": "string[pyarrow]", 
                  "id3": "string[pyarrow]"}).memory_usage_per_partition(deep=True).compute()


format_bytes(mem[0])

'71.53 MiB'